# Analiza danych przestrzennych - ćwiczenia laboratoryjne 2023/2024

Ten notatnik zalicza się do grupy zestawów zadań, na podstawie których odbywa się zaliczenie ćwiczeń i podlega zwrotowi do oceny w ustalonym na zajęciach terminie.

Uwagi i wytyczne ogólne dotyczące uzupełniania i oceny notatnika:
- Podczas wykonywania zadań należy korzystać wyłącznie z pakietów zaimportowanych na początku notatnika oraz z pakietów wchodzących w skład standardowej biblioteki Pythona, które można zaimportować samodzielnie we wskazanej komórce notatnika.
- Swoje rozwiązania należy wprowadzać wyłącznie w miejce następujących fragmentów kodu:<br/> `# YOUR CODE HERE`<br/> `raise NotImplementedError()`<br/> 
a odpowiedzi tekstowe w komórkach oznaczonych hasłem:<br/> 
`YOUR ANSWER HERE`<br/> 
Nie należy w żaden sposób modyfikować pozostałych fragmentów kodu oraz innych elementów notatnika, w szczególności dodawać lub usuwać komórek oraz zmieniać nazwy pliku.
- Jeżeli zestaw zadań wymaga skorzystania z fragmentów kodu opracowanego w ramach wcześniejszych zestawów zadań należy je umieścić we wskazanej komórce notatnika.
- Otrzymywane wyniki i odpowiedzi mają być rezultatem wykonania napisanego kodu, odpowiedzi uzupełniane manualnie nie podlegają ocenie.
- Zawarte w notatniku automatyczne testy mają charakter poglądowy. Dotyczą one wybranych aspektów zadań i mają na celu wyłapać podstawowe błędy. Przejście przez kod wszystkich testów nie oznacza, że zadanie jest wykonane w całości poprawnie i zostanie ocenione na maksymalną liczbę punktów.
- Zadania należy wykonać w taki sposób, aby podczas wykonywania kodu nie zostały wyświetlone żadne ostrzeżenia.
- Zadania, które powodują wyświetlenie komunikatu o błędzie przerywającym wykonywanie kodu nie podlegają ocenie.

Uwagi i wytyczne ogólne dotyczące wizualizacji wyników:
- Wszystkie wykresy powinny być wykonane w jednolitym, przejrzystym i czytelnym stylu, posiadać odpowiednio dobrane proporcje i zakresy wartości osi.
- Figury powinny mieć ustawione białe tło, tak, aby niezależnie od ustawień notatnika wszystkie elementy wykresów były dobrze widoczne (domyślnie tło jest przeźroczyste co może powodować problemy w notatnikach z ustawionym ciemnym tłem). Rozmiar poziomy figur nie powinien przekraczać 20 cali.
- Wykresy oraz ich osie powinny mieć nadane tytuły. Jeżeli w obrębie figury znajduje się więcej niż jeden wykres to figura również powinna mieć nadany tytuł.
- Zakresy osi wykresów przedstawiających rozkłady punktów w przestrzeni powinny być docięte do granic obszaru, na którym został wygenerowany proces punktowy.
- Proporcje osi wykresów przedstawiających rozkłady punktów w przestrzeni powinny być dobrane tak, aby wykresy odzwierciedlały rzeczywisty rozkład punktów w przestrzeni. Nie należy osiągać tego efektu manipulując rozmiarem całej figury.

Przed odesłaniem zestawu zadań do oceny proszę uzupełnić komórkę z danymi autorów rozwiązania (nazwa zespołu oraz imiona, nazwiska i numery indeksów członków zespołu) oraz upewnić się, że notatnik działa zgodnie z oczekiwaniami. W tym celu należy skorzystać z opcji **Restart Kernel and Run All Cells...** dostępnej na górnej belce notatnika pod symbolem $\blacktriangleright\blacktriangleright$.

---

## Zestaw zadań 2: Badanie intensywności procesów punktowych (część 1)

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Miejsce do importu pakietów wchodzących w skład standardowej biblioteki Pythona oraz ustawienie opcji wykorzystywanych pakietów

### Wczytanie danych

Załączone do notatniki pliki zawierają następujące procesy punktowe:
 - `points_HP` - jednorodny rozkład Poissona,
 - `points_UP` - niejednorodny rozkład Poissona,
 - `points_M` - rozkład Materna,
 - `points_T` - rozkład Thomasa,
 
wygenerowane dla zakresu współrzędnych X $[-10, 10]$ i Y $[-5, 5]$.

W celu wczytania danych do notatnika umieść wszystkie pliki w tym samym folderze, w którym znajduje się notatnik.

In [7]:
# Wczytanie danych
points_HP = pd.read_pickle('points_HP.pkl')
points_UP = pd.read_pickle('points_UP.pkl')
points_M = pd.read_pickle('points_M.pkl')
points_T = pd.read_pickle('points_T.pkl')

### Zadanie 1: Badanie intensywności procesu punktowego metodą szacowania lokalnego (15 pkt)

Przygotuj funkcję `point_count_on_subregions()`, która będzie zliczać punkty w obrębie prostokątnych podobszarów oraz funkcję `intensity_on_subregions()`, która będzie obliczać intensywność procesu punktowego w obrębie prostokątnych podobszarów.

Algorytm postępowania:
1. Dzielimy analizowany obszar na identyczne prostokątne podobszary ułożone w równomiernej siatce (w wierszach i kolumnach).
2. Zliczamy punkty znajdujące się w poszczególych podobszarach (punkty znajdujące się na granicach podobszarów powinny być zliczone tylko do jednego z nich).
3. Obliczamy intensywność procesu w obrębie poszczególnych podobszarów dzieląc znajdującą się w nich liczbę punktów przez pole powierzchni podobszaru.

Uwagi do wykonania zadania:
- W części zadania dotyczącej funkcji obliczającej intensywność procesu nie należy dublować kodu napisanego w funkcji zliczającej punkty w obrębie prostokątnych podobszarów, a jedynie wywołać przygotowaną wcześniej funkcję `point_count_on_subregions()`.
- W metodzie szacowania lokalnego szacowanie intensywności odbywa się na kwadratowych podobszarach. W celu ułatwienia pisania funkcji warunek ten w zadaniu został rozluźniony do obszarów prostokątnych. We wszystkich zadaniach rozmiar obszaru i parametry siatki podobszarów są natomiast dobrane w taki sposób, że wygenerowane podobszary będą kwadratami.

#### a) Przygotowanie funkcji

In [12]:
def point_count_on_subregions(points, bins, x_lim, y_lim):
    bin_data = []
    x_step = (x_lim[1] - x_lim[0])/bins[0]
    y_step = (y_lim[1] - y_lim[0])/bins[1]
    x = x_lim[0]
    while (x <= x_lim[1] - x_step):
        y = y_lim[0]
        x_start = x
        x_end = x + x_step
        x += x_step
        while (y <= y_lim[1] - y_step):
            y_start = y
            y_end = y + y_step
            y += y_step
            print(x_start, x_end, "\n", y_start, y_end)

point_count_on_subregions(points_HP, bins=[10, 5], x_lim=[-5, 5], y_lim=[-2.5, 2.5])

def intensity_on_subregions(points, bins, x_lim, y_lim):

    """
    Parameters
    -------
    points: DataFrame
        Tablica zawierająca dwie kolumny ze współrzędnymi punktów opisane jako "X" i "Y".
    bins: list
        Lista określająca liczbę podobszarów w poziomie i pionie.
        Przykład: [10, 10]
    x_lim: list
        Lista określająca zakres wartości współrzędnej X.
        Przykład: [0, 10]
    y_lim: list
        Lista określająca zakres wartości współrzędnej Y.
        Przykład: [0, 10]   

    Returns
    -------
    intensity_data: list
        Lista zawierająca trzy macierze:
        - 2D z wartością intensywności przypisą do każdego z podobszarów.
        - 1D ze współrzędnymi krawędzi podobszarów na osi X,
        - 1D ze współrzędnymi krawędzi podobszarów na osi Y,
        Na przykład: [array([[3, 7], [9, 4]]), array([0, 1, 2]), array([0, 1, 2])]
    """
    # YOUR CODE HERE
    raise NotImplementedError()

-5 -4.0 
 -2.5 -1.5
-5 -4.0 
 -1.5 -0.5
-5 -4.0 
 -0.5 0.5
-5 -4.0 
 0.5 1.5
-5 -4.0 
 1.5 2.5
-4.0 -3.0 
 -2.5 -1.5
-4.0 -3.0 
 -1.5 -0.5
-4.0 -3.0 
 -0.5 0.5
-4.0 -3.0 
 0.5 1.5
-4.0 -3.0 
 1.5 2.5
-3.0 -2.0 
 -2.5 -1.5
-3.0 -2.0 
 -1.5 -0.5
-3.0 -2.0 
 -0.5 0.5
-3.0 -2.0 
 0.5 1.5
-3.0 -2.0 
 1.5 2.5
-2.0 -1.0 
 -2.5 -1.5
-2.0 -1.0 
 -1.5 -0.5
-2.0 -1.0 
 -0.5 0.5
-2.0 -1.0 
 0.5 1.5
-2.0 -1.0 
 1.5 2.5
-1.0 0.0 
 -2.5 -1.5
-1.0 0.0 
 -1.5 -0.5
-1.0 0.0 
 -0.5 0.5
-1.0 0.0 
 0.5 1.5
-1.0 0.0 
 1.5 2.5
0.0 1.0 
 -2.5 -1.5
0.0 1.0 
 -1.5 -0.5
0.0 1.0 
 -0.5 0.5
0.0 1.0 
 0.5 1.5
0.0 1.0 
 1.5 2.5
1.0 2.0 
 -2.5 -1.5
1.0 2.0 
 -1.5 -0.5
1.0 2.0 
 -0.5 0.5
1.0 2.0 
 0.5 1.5
1.0 2.0 
 1.5 2.5
2.0 3.0 
 -2.5 -1.5
2.0 3.0 
 -1.5 -0.5
2.0 3.0 
 -0.5 0.5
2.0 3.0 
 0.5 1.5
2.0 3.0 
 1.5 2.5
3.0 4.0 
 -2.5 -1.5
3.0 4.0 
 -1.5 -0.5
3.0 4.0 
 -0.5 0.5
3.0 4.0 
 0.5 1.5
3.0 4.0 
 1.5 2.5
4.0 5.0 
 -2.5 -1.5
4.0 5.0 
 -1.5 -0.5
4.0 5.0 
 -0.5 0.5
4.0 5.0 
 0.5 1.5
4.0 5.0 
 1.5 2.5


In [ ]:
# Komórka testowa
test_intensity_1 = intensity_on_subregions(points_HP, bins=[10, 5], x_lim=[-5, 5], y_lim=[-2.5, 2.5])
assert type(test_intensity_1) == list
assert len(test_intensity_1) == 3
assert np.shape(test_intensity_1[0]) == (5, 10)
assert np.all(np.isclose(np.sum(test_intensity_1[0], axis=0), np.array([47., 38., 56., 45., 54., 51., 46., 57., 50., 66.])))
assert np.all(np.isclose(np.sum(test_intensity_1[0], axis=1), np.array([98.,  96.,  96., 111., 109.])))
assert np.shape(test_intensity_1[1]) == (11,)
assert np.all(np.isclose(test_intensity_1[1], np.array([-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.,  5.])))
assert np.shape(test_intensity_1[2]) == (6,)
assert np.all(np.isclose(test_intensity_1[2], np.array([-2.5, -1.5, -0.5,  0.5,  1.5,  2.5])))

#### b) Wygenerowanie danych i wizualizacja

Wykorzystaj funkcje przygotowane w poprzednim podpunkcie zadania do zbadania intensywności wszystkich wczytanych do notatnika rozkładów punktowych. Ustaw podział obszaru na 40 podobszarów wzdłuż osi $X$ i 20 podobszarów wzdłuż osi $Y$. Analizę przeprowadź dla całego obszaru, na którym zostały wygenerowane punkty.

Przedstaw wyniki analizy graficznie w postaci histogramów dwuwymiarowych z nałożonymi na nie rozkładami punktów. Zestaw wyniki na pojedynczej figurze (siatka wykresów 2x2).
Dla wszystkich histogramów ustaw tę samą skalę kolorów z identycznym zakresem wartości ustalonym na podstawie minimalnej i maksymalnej wartości intensywności wyznaczonej dla kompletu analizowanych rozkładów.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Zadanie 2: Badanie intensywności procesu punktowego metodą funkcji jądrowych (20 pkt)

Przygotuj funkcję `intensity_on_kde()`, która będzie obliczać intensywność procesu punktowego z wykorzystaniem jądrowego estymatora intensywności.

Algorytm postępowania:
1) Generujemy punkty pomiarowe na regularnej siatce tak, że poszczególne punkty są oddalone od siebie o odległość $d_x$ w poziomie i $d_y$ w pionie, a skrajne punkty leżą na granicach analizowanego obszaru.
2) Dla każdego z wygenerowanych punktów pomiarowych szacujemy intensywność procesu za pomocą jądrowego estymatora intensywności danego wzorem:
$$ 	\grave{\lambda}(x,y) = \sum_{d_i < \tau} \frac{3}{\pi \tau^2} (1-\frac{d_i^2}{\tau^2})^2$$ 
gdzie: $d_i$ - odległość punktu od centrum funkcji jądrowej, $\tau$ - promień funkcji jądrowej.

Uwagi do wykonania zadania:
- Podczas pisania funkcji należy wykorzystać maksymalnie jedną pętlę `for` iterującą po kolejnych pozycjach funkcji jądrowej. Nie należy korzystać z pętli `for` podczas wyliczania wartości funkcji jądrowej w poszczególnych jej lokalizacjach.
- Funkcja przygotowana według powyższych wytycznych nie ma wbudowanego mechanizmu korekty na efekt brzegowy, w związku z czym będzie zaniżać wartość intensywności przy krawędziach analizowanego obszaru.

#### a) Przygotowanie funkcji

In [ ]:
def intensity_on_kde(points, kernel_radius, grid, x_lim, y_lim):
    """
    Parameters
    -------
    points: DataFrame
        Tablica zawierająca dwie kolumny ze współrzędnymi punktów opisane jako "X" i "Y".
    kernel_radius: float
        Liczba dodatnia określająca promień funkcji jądrowej.
    grid: list
        Lista określająca liczbę punktów w poziomie i pionie, dla których wyliczane będą wartości intensywności.
        Przykład: [10, 10]
    x_lim: list
        Lista określająca zakres wartości współrzędnej X.
        Przykład: [0, 10]
    y_lim: list
        Lista określająca zakres wartości współrzędnej Y.
        Przykład: [0, 10]   

    Returns
    -------
    intensity_data: DataFrame
        Tablica zawierająca trzy kolumny - dwie ze współrzędnymi punktów, dla których wyliczone zostały wartości intensywności
        opisane jako "X" i "Y" oraz kolumnę z wartościami intensywności wyliczonymi dla tych współrzędnych opisaną jako "I".
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Komórka testowa
test_intensity_2 = intensity_on_kde(points_HP, kernel_radius=1.25, grid=[50, 25], x_lim=[-5, 5], y_lim=[-2.5, 2.5])
assert type(test_intensity_2) == pd.DataFrame
assert test_intensity_2.shape == (1250, 3)
assert list(test_intensity_2.columns) == ["X", "Y", "I"]
assert np.isclose(np.min(test_intensity_2["X"]), -5)
assert np.isclose(np.max(test_intensity_2["X"]), 5)
assert np.isclose(np.min(test_intensity_2["Y"]), -2.5)
assert np.isclose(np.max(test_intensity_2["Y"]), 2.5)
assert np.isclose(np.min(test_intensity_2["I"]), 7.335892819947689)
assert np.isclose(np.max(test_intensity_2["I"]), 20.311900026946358)
assert np.isclose(np.mean(test_intensity_2["I"]), 14.054515271834926)

#### b) Wygenerowanie danych i wizualizacja
Wykorzystaj przygotowaną funkcje do zbadania intensywności wszystkich wczytanych do notatnika rozkładów punktowych. Wykonaj obliczania dla promienia funkcji jądrowej równego 1.5 i siatki 200x100 punktów pomiarowych. Analizę przeprowadź dla całego obszaru, na którym zostały wygenerowane punkty.

Przedstaw wyniki analizy graficznie w postaci wykresów konturowych z wypełnieniem z nałożonymi na nie rozkładami punktów. Zestaw wyniki na pojedynczej figurze (siatka wykresów 2x2). Dla wszystkich histogramów ustaw tę samą skalę kolorów z identycznym zakresem wartości ustalonym na podstawie minimalnej i maksymalnej wartości intensywności wyznaczonej dla kompletu analizowanych rozkładów.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()